In [1]:
import os
from pathlib import Path

from qiskit import transpile
from qiskit.circuit.random import random_circuit
from qiskit.providers.fake_provider import FakeLimaV2
from qiskit.quantum_info import SparsePauliOp
from qiskit.primitives import BackendEstimator, Estimator

from tqdm.notebook import tqdm_notebook

from blackwater.data import ExpValData, ExpValDataWriter

from blackwater.data.utils import get_backend_properties_v1, encode_pauli_sum_op
from blackwater.exception import BlackwaterException
from blackwater.library.temp import encode_data

In [9]:
from qiskit import QuantumCircuit, transpile
from qiskit.opflow import PauliSumOp
from qiskit.primitives import BaseEstimator, EstimatorResult
from qiskit.providers import JobV1 as Job, Options, BackendV2, Backend, BackendV1
from qiskit.quantum_info import SparsePauliOp
from qiskit.providers.fake_provider import FakeLimaV2, FakeLima, ConfigurableFakeBackend
from qiskit.opflow import I, X, Z
from qiskit.algorithms.minimum_eigensolvers import VQE, VQEResult
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal

In [3]:
from blackwater.data.utils import generate_random_pauli_sum_op, get_backend_properties_v1
from blackwater.library.learning.estimator import learning, EmptyProcessor, TorchLearningModelProcessor
from blackwater.library.temp import encode_data, MLP1

In [116]:
from qiskit_aer import AerSimulator
backend_noisy = AerSimulator.from_backend(backend) # Noisy
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [104]:
# backend = ConfigurableFakeBackend("Tashkent", n_qubits=4, version=1,
#                                  # qubit_t1=0.00011377694598809795,
#                                  # qubit_t2=0.00010553074510239035
#                                  )
backend = FakeLima()
properties = get_backend_properties_v1(backend)

In [31]:
# ideal_estimator = Estimator() # AerEstimator(skip_transpilation=True) #

In [37]:
# import itertools
# pauli_list_full = ["".join(s) for s in itertools.product(["X", "Y", "Z", "I"], repeat=4)]
# pauli_list_full

In [32]:
# operator = (-1.052373245772859 * I ^ I) + \
#             (0.39793742484318045 * I ^ Z) + \
#             (-0.39793742484318045 * Z ^ I) + \
#             (-0.01128010425623538 * Z ^ Z) + \
#             (0.18093119978423156 * X ^ X)

# ansatz = TwoLocal(rotation_blocks='rx', entanglement_blocks='cx', reps=3)
# ansatz.num_qubits = operator.num_qubits

In [105]:
import itertools
import numpy as np

np.random.seed(0)
pauli_list_full = [''.join(s) for s in itertools.product(['X','Y','Z','I'], repeat=4)]
np.random.shuffle(pauli_list_full)

sep = 128

train_paulis = pauli_list_full[:sep]
test_paulis = pauli_list_full[sep:]
print(len(train_paulis), len(test_paulis))

128 128


In [110]:
train_circuits = []
N = 10
ansatz = TwoLocal(num_qubits=4, rotation_blocks='rx', entanglement_blocks='cx', reps=3)
num_params = len(list(ansatz.parameters))
for pauli in train_paulis:
    measurement_circ = QuantumCircuit(4)
    for i, p in enumerate(pauli):
        if p in ['Z', 'I']:
            pass
        elif p == 'X':
            measurement_circ.x(i)
        elif p == 'Y':
            measurement_circ.sdg(i)
            measurement_circ.h(i)
        else:
            raise Exception
    ansatz.compose(measurement_circ, inplace=True)
    ansatz.measure_all()
    for _ in range(N):
        train_circuits.append(ansatz.bind_parameters(np.random.uniform(-5, 5, num_params)))

In [111]:
test_circuits = []
N = 10
ansatz = TwoLocal(num_qubits=4, rotation_blocks='rx', entanglement_blocks='cx', reps=3)
num_params = len(list(ansatz.parameters))
for pauli in test_paulis:
    measurement_circ = QuantumCircuit(4)
    for i, p in enumerate(pauli):
        if p in ['Z', 'I']:
            pass
        elif p == 'X':
            measurement_circ.x(i)
        elif p == 'Y':
            measurement_circ.sdg(i)
            measurement_circ.h(i)
        else:
            raise Exception
    ansatz.compose(measurement_circ, inplace=True)
    ansatz.measure_all()
    for _ in range(N):
        test_circuits.append(ansatz.bind_parameters(np.random.uniform(-5, 5, num_params)))

In [98]:
test_circuits[:5]

In [118]:
from qiskit.result import marginal_counts
from qiskit import execute
from qiskit_aer import QasmSimulator

qasm_sim = QasmSimulator()

def get_all_z_exp_wo_shot_noise(circuit, marginal_over=None):
    circuit_copy = circuit.copy()
    circuit_copy.remove_final_measurements()
    circuit_copy.save_density_matrix()

    def int_to_bin(n, num_bits=4):
        if n < 2**num_bits:
            binary_str = bin(n)[2:]
            return binary_str.zfill(num_bits)
        else:
            raise ValueError

    # print(circuit_copy.draw())
    circuit_copy = transpile(circuit_copy, backend=backend_noisy, optimization_level=3)
    job = qasm_sim.run(circuit_copy)
    # job = execute(circuit_copy, QasmSimulator(), backend_options={'method': 'statevector'})
    probs = np.real(np.diag(job.result().results[0].data.density_matrix))
    probs = {int_to_bin(i, num_bits=4): p for i, p in enumerate(probs)}

    if marginal_over:
        probs = marginal_counts(probs, indices=marginal_over)

    exp_val = 0
    for key, prob in probs.items():
        num_ones = key.count('1')
        exp_val += (-1) ** num_ones * prob

    return exp_val

train_ideal_exp_vals = []
for train_circ, obs in zip(train_circuits, train_paulis):
    marginal_over = np.where(obs == 'I')[0].tolist() if 'I' in obs else None
    train_ideal_exp_vals.append(get_all_z_exp_wo_shot_noise(train_circ, marginal_over=marginal_over))

In [119]:
test_ideal_exp_vals = []
for test_circ, obs in zip(test_circuits, test_paulis):
    marginal_over = np.where(obs == 'I')[0].tolist() if 'I' in obs else None
    test_ideal_exp_vals.append(get_all_z_exp_wo_shot_noise(test_circ, marginal_over=marginal_over))

In [120]:
def cal_all_z_exp(counts, marginal_over=None):
    if marginal_over:
        counts = marginal_counts(counts, indices=marginal_over)

    shots = sum(list(counts.values()))
    all_z_exp = 0
    for key, value in counts.items():
        num_ones = key.count('1')
        sign = (-1) ** num_ones  # Sign of the term in 'key' depends on the number of 0's, e.g. '11' is +, '110' is -
        all_z_exp += sign * value
    all_z_exp = all_z_exp / shots

    return all_z_exp

In [ ]:
train_noisy_exp_vals = []
for circuit in tqdm_notebook(train_circuits):
    job_noisy = execute(circuit, **run_config_noisy)
    counts_noisy = job_noisy.result().get_counts()
    train_noisy_exp_vals.append(cal_all_z_exp(counts_noisy, marginal_over=marginal_over))

  0%|          | 0/1280 [00:00<?, ?it/s]

In [ ]:
test_noisy_exp_vals = []
for circuit in tqdm_notebook(test_circuits):
    job_noisy = execute(circuit, **run_config_noisy)
    counts_noisy = job_noisy.result().get_counts()
    test_noisy_exp_vals.append(cal_all_z_exp(counts_noisy, marginal_over=marginal_over))

In [ ]:
train_observables = [encode_pauli_sum_op(SparsePauliOp(basis))[0] for basis in train_paulis]
test_observables = [encode_pauli_sum_op(SparsePauliOp(basis))[0] for basis in test_paulis]
train_noisy_exp_vals_copy = [x[0][0] for x in train_noisy_exp_vals]
test_noisy_exp_vals_copy = [x[0][0] for x in test_noisy_exp_vals]
X_train, y_train = encode_data(train_circuits, properties, train_ideal_exp_vals, train_noisy_exp_vals_copy, num_qubits=1, meas_bases=train_observables)
X_test, y_test = encode_data(test_circuits, properties, test_ideal_exp_vals, test_noisy_exp_vals_copy, num_qubits=1, meas_bases=test_observables)

BATCH_SIZE = 32
fix_random_seed(0)
test_dataset = TensorDataset(torch.Tensor(X_test), torch.Tensor(y_test))
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE*1000, shuffle=False)




In [25]:
backend = ConfigurableFakeBackend("Tashkent", n_qubits=2, version=1)
model = MLP1(64, 5, 1)
backend = FakeLima()
        

In [29]:
processor = TorchLearningModelProcessor(
    model=model,
    backend=backend
)

circuit = transpile(random_circuit(2, 2, measure=False), backend)
obs = generate_random_pauli_sum_op(2, 1, coeff=1.0)

coefficent = np.random.uniform(-1, 1, size=10)
operator = coefficient.dot(np.random.choice(train_paulis, size=2)+np.random.choice(test_paulis, size=8))



ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")


learning_estimator = learning(Estimator, processor=processor, backend=backend, skip_transpile=True)
estimator = learning_estimator()

slsqp = SLSQP(maxiter=100)
vqe = VQE(estimator, ansatz, slsqp, callback=lambda a, b, c, d: print(c))
result = vqe.compute_minimum_eigenvalue(operator)

(-0.38997631156859947+0j)
(-0.38997631156859947+0j)
(-0.38997631156859947+0j)
(-0.38997631156859947+0j)
(-0.38997631156859947+0j)
(-0.38997631156859947+0j)
(-0.38997631156859947+0j)
(-0.38997631156859947+0j)
(-0.38997631156859947+0j)


In [27]:
result.eigenvalue.real

-0.3910270230824997